In [115]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

In [143]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa142.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "k3KohcekO4CQRcy"

In [117]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            if duration_unit == "d":
                barrier = "10000"
            else:
                barrier = "+0"
            msg = {
                "proposal": 1,
                "amount" : 0.0005,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "BTC",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sendbar')
            ws.send(json.dumps(msg))
        else:
            print('here')
            amount = 0.0005
            if 'error' in message.keys():
                barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                if message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    print('amount =',amount)
                    print(barrier)
                    print(contract_type)
                    print(duration)
                    print(duration_unit)
                    print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print('sent')
                ws.send(json.dumps(msg))   
            else:
                return_json.append(message)
                ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [118]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):
    
    spot = float(pd_series.Spot)
    
    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    digits = int( assets[assets['Asset']== symbol].digits )
    
    sigma = float( assets[assets['Asset']==symbol].IV )  
    
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1
    
    spot = round(spot/10**-n_digits)*10**-n_digits
    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400       
    
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if duration_unit != 'd':
        
        deltas = np.arange(1,10,2)/10 
    
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )
    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )
        
    else:
        
        temp_barriers = []
        
        displayed_barriers = []

        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )

        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10

        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 

        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)

        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)

        temp_barriers.append(max_strike)

        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)


    return displayed_barriers
        

In [119]:
def calc_ppp_ask (pd_series, t_step, s_step):

    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    stake = pd_series.Stake

    vol_charge = 1.025
 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400  
    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400

    r = pd_series.Discount_rate 
    
    contract_type = pd_series.Contract_type

    number_of_contracts = pd_series.API_PPP
        
    if s_step == 0:
        
        spot = float(pd_series.Spot)

        strike = pd_series.Barrier
    
    else:
        
        spot = float(tick_history(int(duration[0]), symbol, s_step))
        
        if duration_unit != 'd':
            
            strike = spot + float(pd_series.Displayed_barrier)
            
        else:
            
            strike = pd_series.Barrier
        
    sigma = float( assets[assets['Asset']==symbol].IV )
    
    if duration_unit != 'd':
        
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
        
    else:
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
        
    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))
    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))
    
    return stake / ( bs_price + delta_charge + bs_markup )

In [120]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [121]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [171]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    barrier = 0
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0" 
        print('\n',message)
        print('msgc42re\n\n\n')
        if 'authorize' in message:
            
            msg = {
                "proposal": 1,
                "amount" :0.0005,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "BTC",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))
        else:
            if 'error' in message.keys():
                amount = float(message['error']['details']['min_stake'])
                if message['error']['details']['field'] == 'barrier':
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                    amount = float(message['error']['details']['min_stake'])
                elif message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    barrier = message['echo_req']['barrier']

                print(amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print(msg)
                ws.send(json.dumps(msg))   

            else:
                print('here\n\n\n\n')
                return_json.append(message)
                ws.close()
            
    def on_close(ws, close_status_code, close_msg):
        print()             
 
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    barriers = return_json[-1]['proposal']['barrier_choices']
    
    if 'qa' in master_URL:
        
        return barriers, return_json[-1]['proposal']['pricing_args']['discount_rate']
    
    else:
        
        return barriers, 'prod_api'

In [157]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []

    return_barrier = []
    
    return_discount_rate = []

    contract_ID = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        print('\n',message)
        barrier = 0
        
        amount = 0.0005
        
        message = json.loads(message)
        
#         print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    

        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                amount = float(message['error']['message'].split()[-1][:-1]) 
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)

            barrier = barriers[0][np.random.randint(len(barriers[0]))]

            return_barrier.append(barrier)

            return_discount_rate.append(barriers[1])
            
            print (symbol)
            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "BTC",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre2\n\n\n')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('msgcre3\n\n\n')
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }

            ws.send(json.dumps(msg))
            
        else:
            print('msgcre\n\n\n')
            return_json.append(message)
            
            ws.close()
                                          
    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [133]:
def calc_ppp_bid (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['bid_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400  
        
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if t > 1 / 365:
        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    
    
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['current_spot'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))
        
        if t > 1 / 365:
        
            strike = float(proposal['proposal_open_contract']['barrier'])

        else:

            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

        
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)

    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))

    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts
    

In [125]:
def sell_contract(contract_id):
    
    contract_id = int(contract_id)
    
    temp_json = []
    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]

In [126]:
def calc_ppp_sold (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['sell_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  

    if t > 1 / 365:
        
        r = proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['exit_tick'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)


    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return max(price * number_of_contracts,0)
    


In [127]:
def calc_ppp_exp (proposal, t_step, s_step):
    stake = float(proposal['proposal_open_contract']['sell_price'])
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    if s_step == 0:
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [39]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)
    print(len(barrier_df))
    if len(barrier_df) > 50 - 1:
        break
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        
        max_duration = 1440

    elif duration_unit == 'h':
        
        max_duration = 24

    elif duration_unit == 'd':
        
        max_duration = 365

    i = np.random.randint(5)
    
    if i < 2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1,max_duration)
        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)
    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]
    
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,R_75,VANILLALONGPUT,10729.8339,1689003222,1m,"[1689003222, 1689003282]","[-13.933, -5.7, 0.006, 5.714, 13.962]",0,"{'echo_req': {'amount': 0.006696, 'barrier': '+0.0060', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_75'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.00669600', 'barrier..."
1,R_100,VANILLALONGCALL,10726.57,1689003222,1440m,"[1689003223, 1689089623]","[-680.5, -275.4, 14.7, 312.8, 758.1]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+14.70', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1440, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier..."
2,1HZ50V,VANILLALONGPUT,10268.04,1689003225,207d,"[1689003225, 1706918399]","[6900.0, 7580.0, 8260.0, 8940.0, 9620.0, 10300.0, 11820.0, 13340.0, 14860.0, 16380.0, 17900.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '10300.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 207, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ50V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrie..."
3,1HZ25V,VANILLALONGCALL,9917.23,1689003227,1h,"[1689003227, 1689006827]","[-33.8, -13.8, 0.0, 13.9, 34.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_ch..."
4,1HZ25V,VANILLALONGCALL,9917.31,1689003229,11h,"[1689003229, 1689042829]","[-111.4, -45.5, 0.4, 46.5, 113.4]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.40', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 11, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ25V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_c..."
5,R_50,VANILLALONGCALL,10127.7048,1689003230,1m,"[1689003231, 1689003291]","[-8.771, -3.588, 0.002, 3.594, 8.783]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+0.0020', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_50'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrier_ch..."
6,1HZ100V,VANILLALONGCALL,11973.22,1689003233,1061m,"[1689003233, 1689066893]","[-660.0, -267.6, 12.1, 298.5, 724.1]",0,"{'echo_req': {'amount': 0.01, 'barrier': '+12.10', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'BTC', 'duration': 1061, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ100V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barri..."
7,1HZ50V,VANILLALONGPUT,10271.37,1689003235,188d,"[1689003235, 1705276799]","[7000.0, 7660.0, 8320.0, 8980.0, 9640.0, 10300.0, 11720.0, 13140.0, 14560.0, 15980.0, 17400.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '10300.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 188, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ50V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': '0.01000000', 'barrie..."
8,1HZ10V,VANILLALONGPUT,10086.15,1689003237,182d,"[1689003237, 1704758399]","[9240.0, 9410.0, 9580.0, 9750.0, 9920.0, 10090.0, 10290.0, 10490.0, 10690.0, 10890.0, 11070.0]",0,"{'echo_req': {'amount': 0.01, 'barrier': '10090.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'BTC', 'duration': 182, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': 

In [40]:
Calc_barrier_choices = [] 

for i in barrier_df.index:
    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )

barrier_df.Calc_barrier_choices = Calc_barrier_choices

print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]),\
      '\nMismatches listed below :')

barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]


50 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [72]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    clear_output(wait=True)
    print(len(PPP_df))
    
    if len(PPP_df)>50-1:
        break
        
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
    
    if i<2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1, max_duration)
#     duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
                               

#     clear_output(wait=True)
clear_output(wait=True)

PPP_df


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,1HZ50V,VANILLALONGPUT,10278.4800,1689003401,0.010,1100m,+2.70,10281.1800,"[1689003401, 1689069401]",12479,0,1.025579e-04,0,"{'echo_req': {'contract_id': 12479, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '10281.18', 'barrier_count': 1, 'bid_price': 0.00951903, 'buy_price': '0.01000000', 'contract_id': '12479', 'contract_type': 'VANI..."
1,1HZ100V,VANILLALONGCALL,12011.1000,1689003405,0.010,7h,+4.80,12015.9000,"[1689003405, 1689028605]",12499,0,7.320600e-05,0,"{'echo_req': {'contract_id': 12499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '12015.90', 'barrier_count': 1, 'bid_price': '0.00948590', 'buy_price': '0.01000000', 'contract_id': '12499', 'contract_type': 'VA..."
2,1HZ75V,VANILLALONGPUT,10094.9900,1689003408,0.010,365d,7160.00,7160.0000,"[1689003408, 1720569599]",12519,0,7.384400e-06,0,"{'echo_req': {'contract_id': 12519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '7160.00', 'barrier_count': 1, 'bid_price': 0.00920929, 'buy_price': '0.01000000', 'contract_id': '12519', 'contract_type': 'VANIL..."
3,1HZ10V,VANILLALONGCALL,10083.8500,1689003412,0.010,301m,+0.00,10083.8500,"[1689003412, 1689021472]",12539,0,8.606321e-04,0,"{'echo_req': {'contract_id': 12539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '10083.85', 'barrier_count': 1, 'bid_price': 0.00657065, 'buy_price': '0.01000000', 'contract_id': '12539', 'contract_type': 'VANI..."
4,R_50,VANILLALONGCALL,10134.3614,1689003416,0.010,1440m,-329.7880,9804.5734,"[1689003416, 1689089816]",12559,0,2.846620e-05,0,"{'echo_req': {'contract_id': 12559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '9804.5734', 'barrier_count': 1, 'bid_price': '0.00950640', 'buy_price': '0.01000000', 'contract_id': '12559', 'contract_type': 'V..."
5,R_100,VANILLALONGPUT,10703.1400,1689003520,0.010,1m,+0.00,10703.1400,"[1689003521, 1689003581]",12579,0,1.684784e-03,0,"{'echo_req': {'contract_id': 12579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '10703.14', 'barrier_count': 1, 'bid_price': '0.00951220', 'buy_price': '0.01000000', 'contract_id': '12579', 'contract_type': 'VA..."
6,1HZ25V,VANILLALONGCALL,9909.9900,1689003524,0.010,786m,-49.60,9860.3900,"[1689003524, 1689050684]",12599,0,1.293129e-04,0,"{'echo_req': {'contract_id': 12599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '9860.39', 'barrier_count': 1, 'bid_price': 0.00756186, 'buy_price': '0.01000000', 'contract_id': '12599', 'contract_type': 'VANIL..."
7,1HZ75V,VANILLALONGCALL,10093.0000,1689003528,0.010,400m,+112.40,10205.4000,"[1689003528, 1689027528]",12619,0,2.168505e-04,0,"{'echo_req': {'contract_id': 12619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '10205.40', 'barrier_count': 1, 'bid_price': 0.00704453, 'buy_price': '0.01000000', 'contract_id': '12619', 'contract_type': 'VANI..."
8,1HZ50V,VANILLALONGCALL,10281.3500,1689003532,0.010,498m,+84.70,10366.0500,"[1689003532, 1689033412]",12639,0,3.203580e-04,0,"{'echo_req': {'contract_id': 12639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200501', 'barrier': '10366.05', 'barrier_count': 1, 'bid_price': 0.00911926, 'buy_price': '0.01000000', 'contract_id': '12639', 'contract_type': 'VANI..."
9,R_25,VANILLALONGPUT,10183.0

In [75]:
Calc_ppp = [] 

for i in PPP_df.index:
    
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )

PPP_df.Calc_PPP = Calc_ppp

clear_output(wait = True)

print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]),\
      '\nMismatches listed below :')

df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]

df

50 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json


In [128]:
api_token = 'jjd4dLqBrsJ3eqd'

In [77]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    
    print(len(Bid_df))
    
    if len(Bid_df)>50-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_10'
    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               

    clear_output(wait=True)
Bid_df

50


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ100V,VANILLALONGCALL,12042.1500,1689004092,0.002,22h,+812.00,12854.1500,"[1689004092, 1689083292]",14499,0,6.452120e-05,0,0,0,0,"{'echo_req': {'contract_id': 14499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '12854.15', 'barrier_count': 1, 'bid_price': 0.00165318, 'buy_price': '0.00200000', 'contract_id': '14499', 'contract_type': 'VANI..."
1,R_25,VANILLALONGPUT,10174.8950,1689004094,0.002,833m,+131.440,10306.3350,"[1689004095, 1689054075]",14519,0,1.463760e-05,0,0,0,0,"{'echo_req': {'contract_id': 14519, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10306.335', 'barrier_count': 1, 'bid_price': 0.00198705, 'buy_price': '0.00200000', 'contract_id': '14519', 'contract_type': 'VAN..."
2,R_75,VANILLALONGPUT,10797.9583,1689004098,0.002,286d,8800.0000,8800.0000,"[1689004099, 1713743999]",14539,0,1.159900e-06,0,0,0,0,"{'echo_req': {'contract_id': 14539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '8800.0000', 'barrier_count': 1, 'bid_price': 0.00186485, 'buy_price': '0.00200000', 'contract_id': '14539', 'contract_type': 'VAN..."
3,1HZ75V,VANILLALONGCALL,10135.9300,1689004103,0.002,12h,+152.00,10287.9300,"[1689004103, 1689047303]",14559,0,3.310200e-05,0,0,0,0,"{'echo_req': {'contract_id': 14559, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10287.93', 'barrier_count': 1, 'bid_price': 0.00150691, 'buy_price': '0.00200000', 'contract_id': '14559', 'contract_type': 'VANI..."
4,R_10,VANILLALONGCALL,9910.9570,1689004106,0.002,260d,10140.000,10140.0000,"[1689004106, 1711497599]",14579,0,8.210900e-06,0,0,0,0,"{'echo_req': {'contract_id': 14579, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10140.000', 'barrier_count': 1, 'bid_price': 0.00186643, 'buy_price': '0.00200000', 'contract_id': '14579', 'contract_type': 'VAN..."
5,R_100,VANILLALONGPUT,10680.8600,1689004110,0.002,926m,-551.40,10129.4600,"[1689004110, 1689059670]",14599,0,8.511530e-05,0,0,0,0,"{'echo_req': {'contract_id': 14599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10129.46', 'barrier_count': 1, 'bid_price': 0.00166621, 'buy_price': '0.00200000', 'contract_id': '14599', 'contract_type': 'VANI..."
6,1HZ25V,VANILLALONGCALL,9909.8000,1689004113,0.002,187d,9900.00,9900.0000,"[1689004113, 1705190399]",14619,0,2.742300e-06,0,0,0,0,"{'echo_req': {'contract_id': 14619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '9900.00', 'barrier_count': 1, 'bid_price': 0.00190421, 'buy_price': '0.00200000', 'contract_id': '14619', 'contract_type': 'VANIL..."
7,R_25,VANILLALONGPUT,10177.0130,1689004116,0.002,17h,+0.620,10177.6330,"[1689004117, 1689065317]",14639,0,4.334300e-05,0,0,0,0,"{'echo_req': {'contract_id': 14639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10177.633', 'barrier_count': 1, 'bid_price': '0.00190310', 'buy_price': '0.00200000', 'contract_id': '14639', 'contract_type': 'V..."
8,R_25,VANILLALONGPUT,10177.1190,1689004120,0.002,17h,-142.340,10034.7790,"[1689004121, 1689065321]",14659,0,3.449345e-04,0,0,0,0,"{'echo_req': {'contract_id': 14659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10034.779', 'barrier_count': 1, 'bid_price': 0.00166208, 'buy_price': '0.00200000'

In [78]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in Bid_df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )       
        bid_prop.append(proposal)   
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,8) != 0]
df

50 Test cases 
Mismatches = 5 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
5,R_100,VANILLALONGPUT,10680.860,1689004110,0.002,926m,-551.40,10129.460,"[1689004110, 1689059670]",14599,0,0.000085,1689004308,0.001794,0.001794,"{'echo_req': {'contract_id': 14599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10129.46', 'barrier_count': 1, 'bid_price': 0.00179407, 'buy_price': '0.00200000', 'contract_id': '14599', 'contract_type': 'VANI...","{'echo_req': {'contract_id': 14599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10129.46', 'barrier_count': 1, 'bid_price': 0.00166621, 'buy_price': '0.00200000', 'contract_id': '14599', 'contract_type': 'VANI..."
7,R_25,VANILLALONGPUT,10177.013,1689004116,0.002,17h,+0.620,10177.633,"[1689004117, 1689065317]",14639,0,0.000043,1689004312,0.001880,0.001880,"{'echo_req': {'contract_id': 14639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10177.633', 'barrier_count': 1, 'bid_price': 0.00187964, 'buy_price': '0.00200000', 'contract_id': '14639', 'contract_type': 'VAN...","{'echo_req': {'contract_id': 14639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10177.633', 'barrier_count': 1, 'bid_price': '0.00190310', 'buy_price': '0.00200000', 'contract_id': '14639', 'contract_type': 'V..."
18,R_100,VANILLALONGPUT,10665.840,1689004158,0.002,917m,-548.90,10116.940,"[1689004158, 1689059178]",14859,0,0.000086,1689004330,0.001690,0.001690,"{'echo_req': {'contract_id': 14859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10116.94', 'barrier_count': 1, 'bid_price': 0.00169015, 'buy_price': '0.00200000', 'contract_id': '14859', 'contract_type': 'VANI...","{'echo_req': {'contract_id': 14859, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10116.94', 'barrier_count': 1, 'bid_price': 0.00166567, 'buy_price': '0.00200000', 'contract_id': '14859', 'contract_type': 'VANI..."
22,R_10,VANILLALONGPUT,9911.990,1689004172,0.002,943m,-53.560,9858.430,"[1689004172, 1689060752]",14939,0,0.000917,1689004336,0.001789,0.001789,"{'echo_req': {'contract_id': 14939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '9858.430', 'barrier_count': 1, 'bid_price': 0.00178852, 'buy_price': '0.00200000', 'contract_id': '14939', 'contract_type': 'VANI...","{'echo_req': {'contract_id': 14939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '9858.430', 'barrier_count': 1, 'bid_price': 0.00166202, 'buy_price': '0.00200000', 'contract_id': '14939', 'contract_type': 'VANI..."
43,R_25,VANILLALONGCALL,10178.480,1689004252,0.002,1230m,+160.080,10338.560,"[1689004253, 1689078053]",15359,0,0.000317,1689004370,0.001636,0.001636,"{'echo_req': {'contract_id': 15359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10338.560', 'barrier_count': 1, 'bid_price': 0.00163612, 'buy_price': '0.00200000', 'contract_id': '15359', 'contract_type': 'VAN...","{'echo_req': {'contract_id': 15359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10338.560', 'barrier_count': 1, 'bid_price': 0.00165916, 'buy_price': '0.00200000', 'contract_id': '15359', 'contract_type': 'VAN..."


In [94]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = df.loc[i].bid_prop
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, -1, 0) )       
        bid_prop.append(proposal)   
df.Bid_time = bid_time
df.API_bid = api_bid
df.Calc_bid = calc_bid
df.bid_prop = bid_prop
clear_output(wait = True)
print('Calculating contract value using t -= 1')
print(len(df), 'Test cases','\nMismatches =',\
      len(df[df.API_bid - np.round(df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df2 = df[df.API_bid - np.round(df.Calc_bid,8) != 0]
df2

Calculating contract value using t -= 1
5 Test cases 
Mismatches = 1 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
5,R_100,VANILLALONGPUT,10680.86,1689004110,0.002,926m,-551.40,10129.46,"[1689004110, 1689059670]",14599,0,0.000085,1689004308,0.001794,0.001794,"{'echo_req': {'contract_id': 14599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10129.46', 'barrier_count': 1, 'bid_price': 0.00179407, 'buy_price': '0.00200000', 'contract_id': '14599', 'contract_type': 'VANI...","{'echo_req': {'contract_id': 14599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200521', 'barrier': '10129.46', 'barrier_count': 1, 'bid_price': 0.00166621, 'buy_price': '0.00200000', 'contract_id': '14599', 'contract_type': 'VANI..."


In [95]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in df2.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = df2.loc[i].bid_prop
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, -2 , 0) )       
        bid_prop.append(proposal)   
df2.Bid_time = bid_time
df2.API_bid = api_bid
df2.Calc_bid = calc_bid
df2.bid_prop = bid_prop
clear_output(wait = True)
print('Calculating contract value using t -= 2')
print(len(df2), 'Test cases','\nMismatches =',\
      len(df2[df2.API_bid - np.round(df2.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df22 = df2[df2.API_bid - np.round(df2.Calc_bid,8) != 0]
df22

Calculating contract value using t -= 2
1 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json


In [144]:
api_token = 'UvVjXnvBRUkpjKA'

In [172]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

while True:
    
    print(len(Sold_df))
    
    if len(Sold_df) > 50 - 1:
        break
    
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
    asset = 'R_100'
    duration_unit = random.choice(['m','h','d'])

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), proposal]
                               

    clear_output(wait=True)
    
Sold_df

50


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json
0,R_100,VANILLALONGCALL,10221.77,1689148312,0.000500,728m,+505.70,10727.47,"[1689148314, 1689191994]",15239,0,2.096810e-05,"{'echo_req': {'contract_id': 15239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10727.47', 'barrier_count': 1, 'bid_price': 0.00024633, 'buy_price': '0.00050000', 'contract_id': '15239', 'contract_type': 'VANI..."
1,R_100,VANILLALONGPUT,10223.52,1689148318,0.000500,20h,-241.00,9982.52,"[1689148318, 1689220318]",15259,0,4.801400e-06,"{'echo_req': {'contract_id': 15259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '9982.52', 'barrier_count': 1, 'bid_price': 0.00041791, 'buy_price': '0.00050000', 'contract_id': '15259', 'contract_type': 'VANIL..."
2,R_100,VANILLALONGPUT,10221.03,1689148322,0.000500,1212m,+659.90,10880.93,"[1689148322, 1689221042]",15279,0,7.253000e-07,"{'echo_req': {'contract_id': 15279, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10880.93', 'barrier_count': 1, 'bid_price': 0.00049059, 'buy_price': '0.00050000', 'contract_id': '15279', 'contract_type': 'VANI..."
3,R_100,VANILLALONGPUT,10212.81,1689148326,0.000500,1285m,-248.90,9963.91,"[1689148326, 1689225426]",15299,0,4.651200e-06,"{'echo_req': {'contract_id': 15299, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '9963.91', 'barrier_count': 1, 'bid_price': 0.00041916, 'buy_price': '0.00050000', 'contract_id': '15299', 'contract_type': 'VANIL..."
4,R_100,VANILLALONGPUT,10208.55,1689148328,0.000500,222d,7140.00,7140.00,"[1689148330, 1708387199]",15319,0,3.542000e-07,"{'echo_req': {'contract_id': 15319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '7140.00', 'barrier_count': 1, 'bid_price': 0.00046071, 'buy_price': '0.00050000', 'contract_id': '15319', 'contract_type': 'VANIL..."
5,R_100,VANILLALONGCALL,10203.29,1689148332,0.000500,23h,-635.70,9567.59,"[1689148333, 1689231133]",15339,0,7.489000e-07,"{'echo_req': {'contract_id': 15339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '9567.59', 'barrier_count': 1, 'bid_price': 0.00048928, 'buy_price': '0.00050000', 'contract_id': '15339', 'contract_type': 'VANIL..."
6,R_100,VANILLALONGCALL,10211.46,1689148636,0.000500,14h,+544.80,10756.26,"[1689148637, 1689199037]",15899,0,1.984360e-05,"{'echo_req': {'contract_id': 15899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10756.26', 'barrier_count': 1, 'bid_price': 0.00025815, 'buy_price': '0.00050000', 'contract_id': '15899', 'contract_type': 'VANI..."
7,R_100,VANILLALONGCALL,10212.53,1689148640,0.000500,12h,-189.20,10023.33,"[1689148641, 1689191841]",15919,0,1.850100e-06,"{'echo_req': {'contract_id': 15919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10023.33', 'barrier_count': 1, 'bid_price': 0.00047193, 'buy_price': '0.00050000', 'contract_id': '15919', 'contract_type': 'VANI..."
8,R_100,VANILLALONGCALL,10210.00,1689148644,0.000500,22h,+690.20,10900.20,"[1689148645, 1689227845]",15939,0,1.645240e-05,"{'echo_req': {'contract_id': 15939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10900.20', 'barrier_count': 1, 'bid_price': 0.00028453, 'buy_price': '0.00050000', 'contract_id': '15939', 'contract_type': 'VANI..."
9,R_100,VANILLALONGPUT,10209.50,1689148648,0.000500,1202m,-241.20,

In [173]:
calc_sold = []
sold_time = []
api_sold = []
sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,8) != 0]
df

50 Test cases 
Mismatches = 11 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json,Sold_time,API_sold,Calc_sold,Sold_prop
8,R_100,VANILLALONGCALL,10210.00,1689148644,0.0005,22h,+690.20,10900.20,"[1689148645, 1689227845]",15939,0,1.645240e-05,"{'echo_req': {'contract_id': 15939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10900.20', 'barrier_count': 1, 'bid_price': 0.00028453, 'buy_price': '0.00050000', 'contract_id': '15939', 'contract_type': 'VANI...",1689150520,0.000283,0.000283,"{'echo_req': {'contract_id': 15939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'audit_details': {'contract_start': [{'epoch': 1689148640, 'tick': '10212.53', 'tick_display_value': '10212.53'}, {'epoch': 1689148642, 'tick..."
12,R_100,VANILLALONGPUT,10222.29,1689150276,0.0005,1381m,-636.00,9586.29,"[1689150276, 1689233136]",16419,0,1.555460e-05,"{'echo_req': {'contract_id': 16419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '9586.29', 'barrier_count': 1, 'bid_price': 0.00029448, 'buy_price': '0.00050000', 'contract_id': '16419', 'contract_type': 'VANIL...",1689150534,0.000288,0.000288,"{'echo_req': {'contract_id': 16419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'audit_details': {'contract_start': [{'epoch': 1689150272, 'tick': '10221.88', 'tick_display_value': '10221.88'}, {'epoch': 1689150274, 'tick..."
15,R_100,VANILLALONGPUT,10216.14,1689150286,0.0005,18h,+10.50,10226.64,"[1689150288, 1689215088]",16539,0,2.512000e-06,"{'echo_req': {'contract_id': 16539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10226.64', 'barrier_count': 1, 'bid_price': 0.00045537, 'buy_price': '0.00050000', 'contract_id': '16539', 'contract_type': 'VANI...",1689150544,0.000449,0.000449,"{'echo_req': {'contract_id': 16539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'audit_details': {'contract_start': [{'epoch': 1689150284, 'tick': '10219.3', 'tick_display_value': '10219.30'}, {'epoch': 1689150286, 'flag'..."
24,R_100,VANILLALONGCALL,10228.36,1689150324,0.0005,1371m,+704.20,10932.56,"[1689150324, 1689232584]",16739,0,1.610650e-05,"{'echo_req': {'contract_id': 16739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10932.56', 'barrier_count': 1, 'bid_price': 0.00028734, 'buy_price': '0.00050000', 'contract_id': '16739', 'contract_type': 'VANI...",1689150574,0.000253,0.000253,"{'echo_req': {'contract_id': 16739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'audit_details': {'contract_start': [{'epoch': 1689150320, 'tick': '10228.31', 'tick_display_value': '10228.31'}, {'epoch': 1689150322, 'tick..."
25,R_100,VANILLALONGPUT,10226.92,1689150328,0.0005,12h,+7.00,10233.92,"[1689150328, 1689193528]",16759,0,3.074100e-06,"{'echo_req': {'contract_id': 16759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'barrier': '10233.92', 'barrier_count': 1, 'bid_price': 0.00045035, 'buy_price': '0.00050000', 'contract_id': '16759', 'contract_type': 'VANI...",1689150578,0.000482,0.000482,"{'echo_req': {'contract_id': 16759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200481', 'audit_details': {'contract_start': [{'epoch': 1689150324, 'tick': '10228.36', 'tick_display_value': '10228.36'}, {'epoch': 1689150326, 'tick..."
28,R_100,VANILLALONGCALL,10231.18,1689150340,0.0005,14h,+224.40,10455.58,"[

In [134]:
# Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m')

while True:
    
    print(len(Expired_df))
    
    if len(Expired_df)> 50-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(1, 45)
#     duration=1
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]
                               

    clear_output(wait=True)
Expired_df

50


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,1HZ75V,VANILLALONGPUT,10153.1700,1689005646,0.0020,25m,+68.00,10221.1700,"[1689005647, 1689007147]",16639,0,0.000026,0,0,"{'echo_req': {'contract_id': 16639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10221.17', 'barrier_count': 1, 'bid_price': 0.00174934, 'buy_price': '0.00200000', 'contract_id': '16639', 'contract_type': 'VANI..."
1,1HZ25V,VANILLALONGCALL,9918.7400,1689005650,0.0020,43m,-28.60,9890.1400,"[1689005650, 1689008230]",16659,0,0.000049,0,0,"{'echo_req': {'contract_id': 16659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '9890.14', 'barrier_count': 1, 'bid_price': 0.00092231, 'buy_price': '0.00200000', 'contract_id': '16659', 'contract_type': 'VANIL..."
2,R_25,VANILLALONGPUT,10192.1030,1689005654,0.0020,22m,-21.110,10170.9930,"[1689005654, 1689006974]",16679,0,0.002345,0,0,"{'echo_req': {'contract_id': 16679, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10170.993', 'barrier_count': 1, 'bid_price': 0.00166132, 'buy_price': '0.00200000', 'contract_id': '16679', 'contract_type': 'VAN..."
3,R_100,VANILLALONGCALL,10688.7700,1689005656,0.0020,11m,+25.80,10714.5700,"[1689005657, 1689006317]",16699,0,0.000206,0,0,"{'echo_req': {'contract_id': 16699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10714.57', 'barrier_count': 1, 'bid_price': 0.00171767, 'buy_price': '0.00200000', 'contract_id': '16699', 'contract_type': 'VANI..."
4,R_75,VANILLALONGPUT,10792.4434,1689005660,0.0020,11m,+19.5130,10811.9564,"[1689005661, 1689006321]",16719,0,0.000064,0,0,"{'echo_req': {'contract_id': 16719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10811.9564', 'barrier_count': 1, 'bid_price': 0.00141775, 'buy_price': '0.00200000', 'contract_id': '16719', 'contract_type': 'VA..."
5,R_75,VANILLALONGCALL,10793.7399,1689005664,0.0020,10m,-18.4540,10775.2859,"[1689005665, 1689006265]",16739,0,0.000067,0,0,"{'echo_req': {'contract_id': 16739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10775.2859', 'barrier_count': 1, 'bid_price': 0.00139279, 'buy_price': '0.00200000', 'contract_id': '16739', 'contract_type': 'VA..."
6,R_25,VANILLALONGCALL,10189.4960,1689005668,0.0020,43m,+0.030,10189.5260,"[1689005669, 1689008249]",16759,0,0.000213,0,0,"{'echo_req': {'contract_id': 16759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10189.526', 'barrier_count': 1, 'bid_price': 0.00190228, 'buy_price': '0.00200000', 'contract_id': '16759', 'contract_type': 'VAN..."
7,R_100,VANILLALONGPUT,10684.1100,1689005672,0.0020,13m,+68.50,10752.6100,"[1689005672, 1689006452]",16779,0,0.000028,0,0,"{'echo_req': {'contract_id': 16779, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10752.61', 'barrier_count': 1, 'bid_price': 0.00198678, 'buy_price': '0.00200000', 'contract_id': '16779', 'contract_type': 'VANI..."
8,R_50,VANILLALONGPUT,10135.7928,1689005676,0.0020,36m,+0.0860,10135.8788,"[1689005676, 1689007836]",16799,0,0.000081,0,0,"{'echo_req': {'contract_id': 16799, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': '200541', 'barrier': '10135.8788', 'barrier_count': 1, 'bid_price': 0.00071737, 'buy_price': '0.00200000', 'contract_id': '16799', 'contract_type': 'VA..."
9,1HZ75V,VANILLALONGPUT,10147.0800,1689005

In [140]:
calc_exp = [] 

exp_time = []

api_exp = []

for i in Expired_df.index:
    
    if i >= len(exp_time):
        
        clear_output(wait = True)
                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)
        
        if proposal['proposal_open_contract']['is_sold'] == 1:
            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])

            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))

            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )
            
        else:
            
            exp_time.append(0)
            
            api_exp.append(0)

            calc_exp.append(0)
    
Expired_df['Exp_time'] = exp_time

Expired_df['API_exp'] = api_exp

Expired_df['Calc_exp'] = calc_exp

clear_output(wait = True)

print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,8) != 0]),\
      '\nMismatches listed below :')

df = Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,8) != 0]

df

50 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
